In [1]:
#import sys
import random
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, LSTM
from tensorflow.python.keras import regularizers
from keras.optimizers import Adam
from sklearn.metrics import classification_report
from sklearn.utils import class_weight
from keras.utils import to_categorical
#!{sys.executable} -m pip install keras

Using TensorFlow backend.
/usr/local/Cellar/jupyterlab/3.0.9/libexec/lib/python3.9/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/Cellar/jupyterlab/3.0.9/libexec/lib/python3.9/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/Cellar/jupyterlab/3.0.9/libexec/lib/python3.9/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1

# Preprocessing

In [2]:
"""
    Based on the training data given, we are able to extract 7 attributes:
    1. x accelerometer measurement
    2. y accelerometer measurement
    3. z accelerometer measurement
    4. x gyroscope measurement
    5. y gyroscope measurement
    6. z gyroscope measurement
    7. time stamp for accelerometer and gyroscope measures
    
    We start by creating a dataframe using the csv files provided for readability.
    
    @param x_file: contains the xyz accelerometers and xyz gyroscope measures from the lower limb
    @param x_time_file: contain the time stamps for the accelerometer and gyroscope measures
    @return dataframe of 7 attributes mentioned
"""
def create_dataframe_X(x_file, x_time_file):
    df1 = pd.read_csv(x_file, sep = ',', names = ['X_acc', 'Y_acc', 'Z_acc', 'X_gyr', 'Y_gyr', 'Z_gyr'])
    df1 = scale_data(df1, ['X_acc', 'Y_acc', 'Z_acc', 'X_gyr', 'Y_gyr', 'Z_gyr'])
    df2 = pd.read_csv(x_time_file, names = ['Time stamp'])
    frames = [df1, df2]
    result = pd.concat(frames, axis = 1)
    return result

"""
    Scale the values of X to make it robust to outliers.
    
    @param df: input dataframe
    @param columns: columns to scale
    @return scaled dataframe
"""
def scale_data(df, columns):
    scaler = RobustScaler()
    scaler = scaler.fit(df[columns])
    df.loc[:, columns] = scaler.transform(df[columns])
    return df
    
"""
    We have both the labels and the time stamps for the labels. We create a dataframe from these for
    readability.
    
    @param y_file: contain the labels: 
        (0) indicates standing or walking in solid ground, 
        (1) indicates going down the stairs, 
        (2) indicates going up the stairs, and 
        (3) indicates walking on grass
    @param y_time_file: contain the time stamps for the labels
    @return dataframe of labels and time stamps
""" 
def create_dataframe_Y(y_file, y_time_file):
    df1 = pd.read_csv(y_file, names = ['Label'])
    df2 = pd.read_csv(y_time_file, names = ['Time stamp'])
    frames = [df1, df2]
    result = pd.concat(frames, axis = 1)
    return result
    
"""
    We take the outputs of create_dataframe_X and create_dataframe_Y. In order to combine both of these
    dataframes, we need look at the time intervals present for when the labels were assigned. We down-sample
    the X to the shape of the y.
    
    @param x_frame: dataframe from create_dataframe_X
    @param y_frame: dataframe from create_dataframe_Y
    @return dataframe with 9 columns (8 attributes and 1 label)
"""
def combine_frames(x_frame, y_frame):
    # Change each dataframe column to a list for iterations
    time_stamp_y = y_frame['Time stamp'].tolist()
    time_stamp_x = x_frame['Time stamp'].tolist()
    
    x_range = [] # Empty list to append data points to
    x_random_row = 0 # Initializing variable to hold randomly selected row instance
    refs = []
    count = 0
    for i in range(0, len(time_stamp_y)):
        while (time_stamp_x[count] <= time_stamp_y[i]) and (count <= len(time_stamp_x)):
            x_range.append(time_stamp_x.index(time_stamp_x[count]))
            count += 1
        x_random_row = random.choice(x_range) # Pick a random value
        refs.append(x_random_row) # Keep record of selected rows
        x_range.clear() # Clear the cache
        continue
    
    # Create a new dataframe based on the refs collected - should be roughly the same length as the y_frame
    entries = []
    for item in refs:
        entry = x_frame.iloc[item]
        entries.append(entry)
    
    found_df = pd.concat(entries, axis = 1)
    found_df = found_df.transpose()
    
    # Combine found_df with y_frame for downsampling
    found_df = found_df.reset_index()
    found_df = found_df.drop(['index'], axis = 1)
    found_df = found_df.drop(['Time stamp'], axis = 1)
    combined_frame = pd.concat([found_df, y_frame], axis = 1)
    return combined_frame

"""
    Takes in the sequential X and y and creates windows of time-series data.
    
    @param X: input data
    @param y: label data
    @param time_steps: determines size of window
    @param step: incremental value that window will slide over
    @return time series of X and y data
"""
def mode_labels(X, y, time_steps, step):
    X_values = []
    y_values = []
    for i in range(0, len(X) - time_steps, step):
        value = X.iloc[i:(i + time_steps)].values
        labels = y.iloc[i: (i + time_steps)]
        X_values.append(value)
        y_values.append(stats.mode(labels)[0][0])
    return np.array(X_values), np.array(y_values).reshape(-1, 1)

"""
    Generating data frames from training data.
    
    @param X_file: list of input X files
    @param X_t_file: list of input X_time files
    @param y_file: list of input y files
    @param y_t file: list of y_time files
    @return stacked window of instances across all training files, stack window of labels across all label files
"""
def generate_data(X_file, X_t_file, y_file, y_t_file):
    all_X = []
    all_y = []
    for item_X, item_X_t, item_y, item_y_t in zip(X_file, X_t_file, y_file, y_t_file):
        df_x = create_dataframe_X(item_X, item_X_t)
        df_y = create_dataframe_Y(item_y, item_y_t)
        combined_frame = combine_frames(df_x, df_y)
        X_temp = combined_frame[['X_acc', 'Y_acc', 'Z_acc', 'X_gyr', 'Y_gyr', 'Z_gyr']]
        y_temp = combined_frame['Label']
        X, y = mode_labels(X_temp, y_temp, 30, 1)
        all_X.append(X)
        all_y.append(y)
    return np.concatenate(all_X), np.concatenate(all_y)

In [3]:
# List of training X_files
X_files = ['TrainingData/subject_001_01__x.csv', 'TrainingData/subject_001_02__x.csv', 
           'TrainingData/subject_001_03__x.csv', 'TrainingData/subject_001_04__x.csv',
           'TrainingData/subject_001_05__x.csv', 'TrainingData/subject_001_06__x.csv',
           'TrainingData/subject_001_07__x.csv', 'TrainingData/subject_001_08__x.csv',
           'TrainingData/subject_002_01__x.csv', 'TrainingData/subject_002_02__x.csv',
           'TrainingData/subject_002_03__x.csv', 'TrainingData/subject_001_04__x.csv',
           'TrainingData/subject_002_05__x.csv', 'TrainingData/subject_003_01__x.csv',
           'TrainingData/subject_005_01__x.csv', 'TrainingData/subject_005_02__x.csv',
           'TrainingData/subject_005_03__x.csv', 'TrainingData/subject_006_01__x.csv',
           'TrainingData/subject_006_02__x.csv', 'TrainingData/subject_006_03__x.csv',
           'TrainingData/subject_007_01__x.csv', 'TrainingData/subject_007_02__x.csv',
           'TrainingData/subject_007_03__x.csv', 'TrainingData/subject_007_04__x.csv',
           'TrainingData/subject_008_01__x.csv']

# List of training X_t_files
X_t_files = ['TrainingData/subject_001_01__x_time.csv', 'TrainingData/subject_001_02__x_time.csv', 
             'TrainingData/subject_001_03__x_time.csv', 'TrainingData/subject_001_04__x_time.csv',
             'TrainingData/subject_001_05__x_time.csv', 'TrainingData/subject_001_06__x_time.csv',
             'TrainingData/subject_001_07__x_time.csv', 'TrainingData/subject_001_08__x_time.csv',
             'TrainingData/subject_002_01__x_time.csv', 'TrainingData/subject_002_02__x_time.csv',
             'TrainingData/subject_002_03__x_time.csv', 'TrainingData/subject_001_04__x_time.csv',
             'TrainingData/subject_002_05__x_time.csv', 'TrainingData/subject_003_01__x_time.csv',
             'TrainingData/subject_005_01__x_time.csv', 'TrainingData/subject_005_02__x_time.csv',
             'TrainingData/subject_005_03__x_time.csv', 'TrainingData/subject_006_01__x_time.csv',
             'TrainingData/subject_006_02__x_time.csv', 'TrainingData/subject_006_03__x_time.csv',
             'TrainingData/subject_007_01__x_time.csv', 'TrainingData/subject_007_02__x_time.csv',
             'TrainingData/subject_007_03__x_time.csv', 'TrainingData/subject_007_04__x_time.csv',
             'TrainingData/subject_008_01__x_time.csv']

# List of training y_files
y_files = ['TrainingData/subject_001_01__y.csv', 'TrainingData/subject_001_02__y.csv', 
           'TrainingData/subject_001_03__y.csv', 'TrainingData/subject_001_04__y.csv',
           'TrainingData/subject_001_05__y.csv', 'TrainingData/subject_001_06__y.csv',
           'TrainingData/subject_001_07__y.csv', 'TrainingData/subject_001_08__y.csv',
           'TrainingData/subject_002_01__y.csv', 'TrainingData/subject_002_02__y.csv',
           'TrainingData/subject_002_03__y.csv', 'TrainingData/subject_001_04__y.csv',
           'TrainingData/subject_002_05__y.csv', 'TrainingData/subject_003_01__y.csv',
           'TrainingData/subject_005_01__y.csv', 'TrainingData/subject_005_02__y.csv',
           'TrainingData/subject_005_03__y.csv', 'TrainingData/subject_006_01__y.csv',
           'TrainingData/subject_006_02__y.csv', 'TrainingData/subject_006_03__y.csv',
           'TrainingData/subject_007_01__y.csv', 'TrainingData/subject_007_02__y.csv',
           'TrainingData/subject_007_03__y.csv', 'TrainingData/subject_007_04__y.csv',
           'TrainingData/subject_008_01__y.csv']

# List of training y_t_files
y_t_files = ['TrainingData/subject_001_01__y_time.csv', 'TrainingData/subject_001_02__y_time.csv', 
             'TrainingData/subject_001_03__y_time.csv', 'TrainingData/subject_001_04__y_time.csv',
             'TrainingData/subject_001_05__y_time.csv', 'TrainingData/subject_001_06__y_time.csv',
             'TrainingData/subject_001_07__y_time.csv', 'TrainingData/subject_001_08__y_time.csv',
             'TrainingData/subject_002_01__y_time.csv', 'TrainingData/subject_002_02__y_time.csv',
             'TrainingData/subject_002_03__y_time.csv', 'TrainingData/subject_001_04__y_time.csv',
             'TrainingData/subject_002_05__y_time.csv', 'TrainingData/subject_003_01__y_time.csv',
             'TrainingData/subject_005_01__y_time.csv', 'TrainingData/subject_005_02__y_time.csv',
             'TrainingData/subject_005_03__y_time.csv', 'TrainingData/subject_006_01__y_time.csv',
             'TrainingData/subject_006_02__y_time.csv', 'TrainingData/subject_006_03__y_time.csv',
             'TrainingData/subject_007_01__y_time.csv', 'TrainingData/subject_007_02__y_time.csv',
             'TrainingData/subject_007_03__y_time.csv', 'TrainingData/subject_007_04__y_time.csv',
             'TrainingData/subject_008_01__y_time.csv']

# Use some files to create validation set
val_X = ['TrainingData/subject_003_02__x.csv', 'TrainingData/subject_003_03__x.csv',
         'TrainingData/subject_004_01__x.csv', 'TrainingData/subject_004_02__x.csv',]
val_X_t = ['TrainingData/subject_003_02__x_time.csv', 'TrainingData/subject_003_03__x_time.csv',
           'TrainingData/subject_004_01__x_time.csv', 'TrainingData/subject_004_02__x_time.csv',]
val_y = ['TrainingData/subject_003_02__y.csv', 'TrainingData/subject_003_03__y.csv',
         'TrainingData/subject_004_01__y.csv', 'TrainingData/subject_004_02__y.csv',]
val_y_t = ['TrainingData/subject_003_02__y_time.csv', 'TrainingData/subject_003_03__y_time.csv',
           'TrainingData/subject_004_01__y_time.csv', 'TrainingData/subject_004_02__y_time.csv',]

# TODO: Create the test set
#test_X = ['TestData/subject_009_01__x.csv', 'TestData/subject_010_01__x.csv',
#          'TestData/subject_011_01__x.csv', 'TestData/subject_012_01__x.csv']
#test_X_t = ['TestData/subject_009_01__x_time.csv', 'TestData/subject_010_01__x_time.csv',
#           'TestData/subject_011_01__x_time.csv', 'TestData/subject_012_01__x_time.csv']

training_X, training_y = generate_data(X_files, X_t_files, y_files, y_t_files)
val_X, val_y = generate_data(val_X, val_X_t, val_y, val_y_t)
print(training_X.shape, training_y.shape)
print(val_X.shape, val_y.shape)

(301655, 30, 6) (301655, 1)
(33762, 30, 6) (33762, 1)


In [4]:
label_weights = class_weight.compute_class_weight('balanced', np.unique(training_y), training_y.ravel())
print(label_weights)
label_weights = {i:label_weights[i] for i in range(len(label_weights))} # Create dictionary
print(label_weights)

[0.33593218 5.81133929 4.44263623 1.59734284]
{0: 0.3359321754546953, 1: 5.811339292594591, 2: 4.442636229749632, 3: 1.5973428365669744}


/usr/local/Cellar/jupyterlab/3.0.9/libexec/lib/python3.9/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classes=[0 1 2 3], y=[0 0 0 ... 0 0 0] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [5]:
encoder = OneHotEncoder(handle_unknown = 'ignore', sparse = False)
encoder = encoder.fit(training_y)
training_y_encoded = encoder.transform(training_y)
val_y_encoded = encoder.transform(val_y)

In [6]:
n_timesteps, n_features, n_outputs = training_X.shape[1], training_X.shape[2], training_y_encoded.shape[1]

def define_LSTM_model():
    model = Sequential()
    model.add(LSTM(units = 100, input_shape = (n_timesteps, n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(units = 100, activation = 'relu'))
    model.add(Dense(units = n_outputs, activation = 'softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model
    
def evaluate_model(training_X, training_y_encoded, val_X, val_y_encoded):
    verbose, epochs, batch_size = 1, 15, 64
    model = define_LSTM_model()
    # Fit network
    model.fit(training_X, training_y_encoded, epochs = epochs, batch_size = batch_size, 
              validation_data = (val_X, val_y_encoded), class_weight = label_weights, verbose = verbose)
    # Evaluate model
    _, accuracy = model.evaluate(val_X, val_y_encoded, batch_size = batch_size, verbose = verbose)
    return accuracy

accuracy = evaluate_model(training_X, training_y_encoded, val_X, val_y_encoded)
print(accuracy)

Train on 301655 samples, validate on 33762 samples
Epoch 1/15
301655/301655 [==============================] - 250s 830us/step - loss: 0.3490 - acc: 0.7632 - val_loss: 0.3825 - val_acc: 0.8343
Epoch 2/15
301655/301655 [==============================] - 188s 622us/step - loss: 0.1871 - acc: 0.8710 - val_loss: 0.4841 - val_acc: 0.7920
Epoch 3/15
301655/301655 [==============================] - 394s 1ms/step - loss: 0.1343 - acc: 0.9044 - val_loss: 0.5184 - val_acc: 0.7958
Epoch 4/15
301655/301655 [==============================] - 180s 598us/step - loss: 0.1022 - acc: 0.9263 - val_loss: 0.3183 - val_acc: 0.8715
Epoch 5/15
301655/301655 [==============================] - 184s 610us/step - loss: 0.0825 - acc: 0.9417 - val_loss: 0.6074 - val_acc: 0.8100
Epoch 6/15
301655/301655 [==============================] - 219s 727us/step - loss: 0.0690 - acc: 0.9517 - val_loss: 0.4451 - val_acc: 0.8388
Epoch 7/15
301655/301655 [==============================] - 175s 581us/step - loss: 0.0555 - acc: 0